In [97]:
from langchain import OpenAI
import os

In [98]:
os.environ['OPENAI_API_KEY'] = api_key
llm = OpenAI(temperature = 0.1)
poem = llm("Write a poem on my love for dosa")
print(poem)



Oh dosa, my love for you runs deep
A perfect blend of flavor and heat
Crispy edges, soft center
You are a culinary adventure

From the streets of South India
To the fancy restaurants in America
You have captured hearts and taste buds
With your simple yet delicious ingredients

Rice and lentils, a match made in heaven
Fermented to perfection, a taste so divine
Served with chutney and sambar
You are a symphony of flavors

Oh how I crave your golden hue
Topped with onions, potatoes or cheese too
Each bite, a burst of delight
My love for you, an endless fight

I have tried other dishes, but none compare
To the satisfaction that you bring to my affair
With every bite, my heart sings
For you, my love, are the king of all things

You come in different shapes and sizes
But each one, a treat for my senses
Masala, rava, or plain
You are my comfort food, my love, my bane

I could eat you for breakfast, lunch, and dinner
And never get tired, for you are a winner
In my heart, you hold a special p

In [99]:
from langchain_community.utilities import SQLDatabase

db_user = "root"
db_password = "1216Mihi#"
db_host = "127.0.0.1"
db_name = "atliq_tshirts"

db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}",sample_rows_in_table_info = 4)
print(db.table_info)



CREATE TABLE discounts (
	discount_id INTEGER NOT NULL AUTO_INCREMENT, 
	t_shirt_id INTEGER NOT NULL, 
	pct_discount DECIMAL(5, 2), 
	PRIMARY KEY (discount_id), 
	CONSTRAINT discounts_ibfk_1 FOREIGN KEY(t_shirt_id) REFERENCES t_shirts (t_shirt_id), 
	CONSTRAINT discounts_chk_1 CHECK ((`pct_discount` between 0 and 100))
)DEFAULT CHARSET=utf8mb4 COLLATE utf8mb4_0900_ai_ci ENGINE=InnoDB

/*
4 rows from discounts table:
discount_id	t_shirt_id	pct_discount
1	1	10.00
2	2	15.00
3	3	20.00
4	4	5.00
*/


CREATE TABLE t_shirts (
	t_shirt_id INTEGER NOT NULL AUTO_INCREMENT, 
	brand ENUM('Van Huesen','Levi','Nike','Adidas') NOT NULL, 
	color ENUM('Red','Blue','Black','White') NOT NULL, 
	size ENUM('XS','S','M','L','XL') NOT NULL, 
	price INTEGER, 
	stock_quantity INTEGER NOT NULL, 
	PRIMARY KEY (t_shirt_id), 
	CONSTRAINT t_shirts_chk_1 CHECK ((`price` between 10 and 50))
)DEFAULT CHARSET=utf8mb4 COLLATE utf8mb4_0900_ai_ci ENGINE=InnoDB

/*
4 rows from t_shirts table:
t_shirt_id	brand	color	size	pr

In [100]:
from langchain_experimental.sql import SQLDatabaseChain

db_chain = SQLDatabaseChain.from_llm(llm,db, verbose = True)
qns1 = db_chain.run("How many t shirts stock quantity do we have left for nike in small size and white color")



> Entering new SQLDatabaseChain chain...
How many t shirts stock quantity do we have left for nike in small size and white color
SQLQuery:SELECT SUM(stock_quantity) AS total_stock FROM t_shirts WHERE brand = 'Nike' AND size = 'S' AND color = 'White'
SQLResult: [(Decimal('75'),)]
Answer:75
> Finished chain.


In [101]:
qns1

'75'

In [102]:
qns2 = db_chain.run("How much is the price of the inventory for all small size t shirts. Give the answer without dollar mark and without decimal places")
qns2



> Entering new SQLDatabaseChain chain...
How much is the price of the inventory for all small size t shirts. Give the answer without dollar mark and without decimal places
SQLQuery:SELECT SUM(price*stock_quantity) AS total_price FROM t_shirts WHERE size='S'
SQLResult: [(Decimal('17547'),)]
Answer:17547
> Finished chain.


'17547'

In [103]:
qns3 = db_chain.run("If we have to sell all the Levi's t shirts today with discounts applied. How much revenue the store will generate(Post discounts)")



> Entering new SQLDatabaseChain chain...
If we have to sell all the Levi's t shirts today with discounts applied. How much revenue the store will generate(Post discounts)
SQLQuery:SELECT SUM(price - (price * (pct_discount/100))) AS revenue FROM t_shirts INNER JOIN discounts ON t_shirts.t_shirt_id = discounts.t_shirt_id WHERE brand = 'Levi' AND CURDATE() = 'today' LIMIT 5;

OperationalError: (pymysql.err.OperationalError) (1525, "Incorrect DATE value: 'today'")
[SQL: SELECT SUM(price - (price * (pct_discount/100))) AS revenue FROM t_shirts INNER JOIN discounts ON t_shirts.t_shirt_id = discounts.t_shirt_id WHERE brand = 'Levi' AND CURDATE() = 'today' LIMIT 5;]
(Background on this error at: https://sqlalche.me/e/14/e3q8)

In [104]:
sql_code = """
select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from 
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand ='Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
"""

qns3 = db_chain.run(sql_code)



> Entering new SQLDatabaseChain chain...

select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from 
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand ='Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id

SQLQuery:SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue FROM 
(SELECT sum(price*stock_quantity) as total_amount, t_shirt_id FROM t_shirts WHERE brand ='Levi'
GROUP BY t_shirt_id) a LEFT JOIN discounts ON a.t_shirt_id = discounts.t_shirt_id
SQLResult: [(Decimal('19638.200000'),)]
Answer:The total revenue for Levi brand t-shirts after applying discounts is $19,638.20.
> Finished chain.


In [105]:
qns3

'The total revenue for Levi brand t-shirts after applying discounts is $19,638.20.'

In [106]:
qns4 = db_chain.run("SLECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'")



> Entering new SQLDatabaseChain chain...
SLECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'
SQLQuery:SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'
SQLResult: [(Decimal('21541'),)]
Answer:The total revenue for all Levi t-shirts in stock is $21,541.
> Finished chain.


In [107]:
qns4

'The total revenue for all Levi t-shirts in stock is $21,541.'

In [108]:
qns5 = db_chain.run("SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'")



> Entering new SQLDatabaseChain chain...
SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLQuery:SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLResult: [(Decimal('151'),)]
Answer:151
> Finished chain.


In [109]:
qns5

'151'

## Few Shot Learning

In [110]:
few_shots = [
        {'Question' : "How many t shirts stock quantity do we have left for nike in small size and white color",
        'SQLQuery' : "SELECT SUM(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND size = 'S' AND color = 'White'",
        'SQLResult' : "Result of the SQL query",
        'Answer' : qns1},
        {'Question' : "How much is the price of the inventory for all small size t shirts",
        'SQLQuery' : "SELECT SUM(price * stock_quantity) FROM t_shirts WHERE size = 'S'",
        'SQLResult' : "Result of the SQL query",
        'Answer' : qns2},
        {'Question' : "If we have to sell all the Levi's t shirts today with discounts applied. How much revenue the store will generate(Post discounts)",
        'SQLQuery' : "select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from (select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand ='Levi' group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id",
        'SQLResult' : "Result of the SQL query",
        'Answer' : qns3},
        {'Question' : "If we have to sell all the Levi's T-shirts today, How much revenue our store will get",
        'SQLQuery' : "SLECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'",
        'SQLResult' : "Result of the SQL query",
        'Answer' : qns4},
        {'Question' : "How much white color Levi's shirt I have?",
        'SQLQuery' : "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'",
        'SQLResult' : "Result of the SQL query",
        'Answer' : qns5}
]

In [111]:
type(few_shots)

list

In [112]:
str(few_shots[0].values)

'<built-in method values of dict object at 0x0000024B2FEDAE80>'

In [114]:
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-V2')

In [115]:
to_vectorize = [f"{item['Question']} {item['SQLQuery']} {item['SQLResult']} {item['Answer']}" for item in few_shots]

In [116]:
to_vectorize[0]

"How many t shirts stock quantity do we have left for nike in small size and white color SELECT SUM(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND size = 'S' AND color = 'White' Result of the SQL query 75"

In [121]:
from langchain.vectorstores import Chroma

vectorstore = Chroma.from_texts(to_vectorize, embedding = embeddings, metadatas = few_shots)

In [122]:
from langchain.prompts import SemanticSimilarityExampleSelector

example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vectorstore,
    k=2,
)

example_selector.select_examples({"Question":"How many Adidas T shirts I have left in my store"})

[{'Answer': '75',
  'Question': 'How many t shirts stock quantity do we have left for nike in small size and white color',
  'SQLQuery': "SELECT SUM(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND size = 'S' AND color = 'White'",
  'SQLResult': 'Result of the SQL query'},
 {'Answer': '75',
  'Question': 'How many t shirts stock quantity do we have left for nike in small size and white color',
  'SQLQuery': "SELECT SUM(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND size = 'S' AND color = 'White'",
  'SQLResult': 'Result of the SQL query'}]

In [123]:
from langchain.chains.sql_database.prompt import PROMPT_SUFFIX,_mysql_prompt

In [124]:
print(_mysql_prompt)

You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of

In [125]:
from langchain.prompts.prompt import PromptTemplate

example_prompt = PromptTemplate(
    input_variables = ["Question","SQLQuery","SQLResult","Answer",],
    template = "\nQuestion:{Question}\nSQLQuery: {SQLQuery}\nSQLResult: {SQLResult}\nAnswer: {Answer}"
)

In [128]:
from langchain.prompts import  FewShotPromptTemplate

few_shot_prompt = FewShotPromptTemplate(
    example_selector =example_selector,
    example_prompt = example_prompt,
    prefix= _mysql_prompt,
    suffix = PROMPT_SUFFIX,
    input_variables = ["input","table_info","top_k"],
)

In [129]:
new_chain = SQLDatabaseChain.from_llm(llm,db, verbose = True, prompt = few_shot_prompt)

In [130]:
new_chain("How many white color Levi's t shirts do we have")



> Entering new SQLDatabaseChain chain...
How many white color Levi's t shirts do we have
SQLQuery:SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLResult: [(Decimal('151'),)]
Answer:151
> Finished chain.


{'query': "How many white color Levi's t shirts do we have", 'result': '151'}

In [131]:
new_chain("How much is the price of the inventory for all small size t shirts")



> Entering new SQLDatabaseChain chain...
How much is the price of the inventory for all small size t shirts
SQLQuery:select sum(price*stock_quantity) as total_price from t_shirts where size = 'S'
SQLResult: [(Decimal('17547'),)]
Answer:The total price of the inventory for all small size t-shirts is $17,547.
> Finished chain.


{'query': 'How much is the price of the inventory for all small size t shirts',
 'result': 'The total price of the inventory for all small size t-shirts is $17,547.'}

In [132]:
new_chain("If we have to sell all the Levi's t shirts today with discounts applied. How much revenue the store will generate(Post discounts)")



> Entering new SQLDatabaseChain chain...
If we have to sell all the Levi's t shirts today with discounts applied. How much revenue the store will generate(Post discounts)
SQLQuery:select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from (select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand ='Levi' group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
SQLResult: [(Decimal('19638.200000'),)]
Answer:The total revenue for Levi brand t-shirts after applying discounts is $19,638.20.
> Finished chain.


{'query': "If we have to sell all the Levi's t shirts today with discounts applied. How much revenue the store will generate(Post discounts)",
 'result': 'The total revenue for Levi brand t-shirts after applying discounts is $19,638.20.'}